this notebook goverrns the stereospecificity analysis

structure:
.
..
topology
forcefield
mdps|
    |...

md  |
    |a1|
       |index
       |R|
           |run1|
                |<-a1Rr1.tpr
                |->a1Rr1.xtc
           |...
           |run40
       |S|
         |...
    |...
    |a7|
       |...



In [ ]:
#make project structure

In [1]:
import os
hm = "/home/domain/data/kirill/AB/XOP/docking/ss_an/"
os.chdir(hm)
#### fix this: add storing to correct folders
#####babel -ipdbqt a*.pdbqt -opdb a1.pdb -d

In [2]:
d

NameError: name 'd' is not defined

In [ ]:
# run fix_xop_names_after_vina.ipynb for selected poses

rank complexes by proximity to TS
TS:


In [ ]:
# extract geometry from complexes and measure their variation from the ideal
# NAC parameters:
# distance 3.4A
# angle of attack 160+-15 deg
# rotation of phenyl 0+-25 deg

In [3]:
import numpy as np

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    angle = np.arccos(np.dot(v1_u, v2_u))
    if np.isnan(angle):
        if (v1_u == v2_u).all():
            return 0.0
        else:
            return np.pi
    return angle


##### parse pdb

In [24]:
prots = ['a1',
 'a21',
 'a43',
 'a46',
 'a49',
 'a5',
 'a7',]

In [57]:
results = []
for prot in prots:
    for ss in ["R","S"]:
        P = []
        O = []
        O1 = []
        O2 = []
        C8 = []
        C9 = []
        C10 = []
        nucleophileO = []
        nucleophileC = []

        pr = open('/home/domain/data/kirill/AB/XOP/docking/ss_an/' + prot + '.pdbqt','r')

        for line in pr:

            if "OH" in line and "TYR B  33" in line: nucleophileO = [float(line[31:38].strip()),float(line[40:46].strip()),float(line[48:54].strip())]
            if "CZ" in line and "TYR B  33" in line: nucleophileC = [float(line[31:38].strip()),float(line[40:46].strip()),float(line[48:54].strip())]

        pr.close()

        ligs = open('/home/domain/data/kirill/AB/XOP/docking/ss_an/'+ prot + '/' + prot + ss +'-ready.pdb','r')

        for line in ligs:

            if " P " in line and "XOP" in line: P.append([float(line[31:38].strip()),float(line[40:46].strip()),float(line[48:54].strip())])
            if " O " in line and "XOP" in line: O.append([float(line[31:38].strip()),float(line[40:46].strip()),float(line[48:54].strip())])
            if "O1" in line and "XOP" in line: O1.append([float(line[31:38].strip()),float(line[40:46].strip()),float(line[48:54].strip())])
            if "O2" in line and "XOP" in line: O2.append([float(line[31:38].strip()),float(line[40:46].strip()),float(line[48:54].strip())])
            if "C8" in line and "XOP" in line: C8.append([float(line[31:38].strip()),float(line[40:46].strip()),float(line[48:54].strip())])
            if "C9" in line and "XOP" in line: C9.append([float(line[31:38].strip()),float(line[40:46].strip()),float(line[48:54].strip())])
            if "C10" in line and "XOP" in line: C10.append([float(line[31:38].strip()),float(line[40:46].strip()),float(line[48:54].strip())])

        ligs.close()

        distance = []
        angle = []
        angle2 = []
        tors = []

        nucleophile = np.subtract(nucleophileO, nucleophileC)

        for at in range(len(P)):

            # distance between nucleophile and phosphorus (should be less that 3.4)

            OP = np.subtract(P[at], nucleophileO)
            PO1 = np.subtract(O1[at], P[at])
            distance.append(np.linalg.norm(OP))

            # angle of attack (should be around 160)
            angle.append(180 - angle_between(nucleophile, OP)*180/3.14)
            
            # angle between attacking and leaving group O-P-OPhNO2
            angle2.append(180 - angle_between(OP, PO1)*180/3.14)

            # angle of phenyl substituent rotation relative to O2-O-C8 plane (should be around 0)
            a = np.subtract(C8[at],O[at])
            b = np.subtract(C8[at],O2[at])   
            phen = np.subtract(C9[at],C10[at])  
            n = np.cross(a,b)
            tors.append(abs(90 - angle_between(phen,n)*180/3.14))
    #    results.append([res for res in zip(distance, angle, angle2, tors, [prot+ss] * len(distance)) if res[0] < 3.5 and res[1] > 120])
        results.append([res for res in zip(distance, angle, angle2, tors, [prot+ss] * len(distance)) if res[0] < 3.5 and res[1] > 120])
print results

[[], [], [], [], [], [], [], [(3.3276521753332338, 133.01817752638692, 86.65689179482122, 64.474887579822905, 'a46S')], [], [], [(3.4929898368017125, 136.48122751353517, 96.794679721477593, 73.728299147114512, 'a5R'), (3.4396396613598919, 131.22076175189716, 78.460928734368849, 72.904920123613635, 'a5R')], [(3.467140031784127, 134.94172222574295, 88.913463692346426, 52.105105350845051, 'a5S'), (3.4974304853706584, 140.2674926026873, 61.025012310104884, 8.2194470721636179, 'a5S'), (3.4143344885936444, 141.98124178476266, 130.97048930338721, 78.10873357252396, 'a5S')], [], []]


In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
x = range(len(result))
plt.plot(x, result)
plt.title('R-S transition')
plt.show()

In [ ]:

# and make complexes with protein
os.chdir(hm + "a5")

for d in os.walk(hm):
    break
d = d[1][:-1]

In [187]:
PREFIX = "/home/domain/kirill/progs/bin/gmx"
mdps = hm + "md/MDPS/"
d = ["a5"]
for ab in d:
    print "here goes " + ab
    os.chdir(hm+ab)
    fs = [f for f in os.listdir(hm+ab) if "ready" in f]
    for f in fs:
        os.chdir(hm + ab)
        pose = f[:-10]
        cplx = pose + "-complex.pdb"
        prot = pose[:-3] + ".pdb"
        print cplx
        !printf "5\n" | /home/domain/kirill/progs/bin/gmx pdb2gmx -f $prot -o $cplx -renum -water tip3p # protonates
        !mv topol* posre* ./$pose
      #  !mkdir $pose
        !sed -i /ENDMDL/d $cplx
        !cat $f >> $cplx # add ligand
        
        !mv $cplx $pose/
        !cp $hm"md/topol.top" $pose/
        !cp $hm"md/xop.itp" $pose/
        os.chdir(hm + ab + "/" + pose)
        
        #### GMX #### 
        !echo 0 | editconf -f $cplx -o conf.gro -c -princ -d 0.8
        !/home/domain/kirill/progs/bin/gmx solvate -cp conf.gro -p topol.top -o box_solvated.gro 
      
        !/home/domain/kirill/progs/bin/gmx grompp -f $mdps"em.mdp" -c box_solvated.gro -p topol.top -o box_eq.tpr 
        !echo "15" | $PREFIX genion -s box_eq.tpr -o box_ionized.gro -pname NA -pq 1 -nname CL -nq -1 -neutral -conc 0.1 -p topol.top
        !$PREFIX grompp -f $mdps"em.mdp" -c box_ionized.gro -o em.tpr
        !rm "./#"*

    

here goes a5
a5S_2-complex.pdb
                  :-) GROMACS - gmx pdb2gmx, VERSION 5.1.2 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra   Sebastian Fritsch 
  Gerrit Groenhof   Christoph Junghans   Anca Hamuraru    Vincent Hindriksen
 Dimitrios Karkoulis    Peter Kasson        Jiri Kraus      Carsten Kutzner  
    Per Larsson      Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff 
   Erik Marklund      Teemu Murtola       Szilard Pall       Sander Pronk   
   Roland Schulz     Alexey Shvetsov     Michael Shirts     Alfons Sijbers  
   Peter Tieleman    Teemu Virolainen  Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2015, The GROMACS develo

### run equil

In [110]:
for a in os.walk(hm):
    break
print a
    

('/home/domain/data/kirill/AB/XOP/docking/ss_an/', ['a1', 'a21', 'a43', 'a46', 'a49', 'a5', 'a7', 'md'], ['a1.pdbqt', 'a21.pdbqt', 'a43.pdbqt', 'a46.pdbqt', 'a49.pdbqt', 'a5.pdbqt', 'a7.pdbqt', 'xopS.pdbqt', 'xopR.pdbqt', 'a1R.log', 'a1S.log', 'a21R.log', 'a21S.log', 'a43R.log', 'a43S.log', 'a46R.log', 'a46S.log', 'a49R.log', 'a49S.log', 'a5R.log', 'a5S.log', 'a7R.log', 'a7S.log', 'do.bash'])


In [109]:
a

('/home/domain/data/kirill/AB/XOP/docking/ss_an/md/MDPS',
 [],
 ['em.mdp', 'nvt.mdp', 'npt.mdp', 'md.mdp', 'cone.mdp', 'pre-qm.mdp'])

### fill the runs

### run

In [177]:
wut

41186

 ### analyse SS

take snapshots every 20fs

In [30]:
a = open("/home/domain/data/kirill/AB/XOP/docking/do.bash")

In [31]:
a.name

'/home/domain/data/kirill/AB/XOP/docking/do.bash'